In [1]:
import sqlite3
import os, sys
import json
import os, sys
sys.path.append('/home/loitg/workspace/backbone/src/')
from utils import Coord, Line
from khoi import  transform, ppp
from loi import createStandardGrid, multiple_transform, centralize, mls, alignGuidelines, gls2arr,\
drawField, arr1gls, arr2gls, GridMap

import numpy as np
sys.path.append('/home/loitg/workspace/test_google/')

from pbprocess import get_document_bounds, FeatureType, readPb, draw_bounds, bound2points
ROOT_DIR = '/home/loitg/workspace/backbone/'

db = sqlite3.connect(os.path.join(ROOT_DIR, 'flaskr.sqlite'), detect_types=sqlite3.PARSE_DECLTYPES)
c = db.cursor()
rs = c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", ['images']);

In [2]:
import cv2
import numpy as np
import itertools
from scipy.interpolate import LinearNDInterpolator
from PIL import Image
from matplotlib import pyplot as plt

import os, sys
sys.path.append('/home/loitg/workspace/backbone/src/')
from utils import Coord, Line
from khoi import  transform, ppp
from loi import createStandardGrid, multiple_transform, centralize, mls, alignGuidelines, gls2arr,\
drawField, arr1gls, arr2gls, GridMap

sys.path.append('/home/loitg/workspace/test_google/')

from pbprocess import get_document_bounds, FeatureType, readPb, draw_bounds, bound2points

In [3]:
rs = c.execute("select * from images")

In [4]:
allrows = rs.fetchall()
imageName_all = [row[0] for row in allrows if row[1] != 'None']
imageStatus_all = [row[1] for row in allrows if row[1] != 'None']
guidelines_all = [json.loads(row[2]) for row in allrows if row[1] != 'None']

def cvtGL2Coords(guidelines):
    return [[Coord(a["x"], a["y"]) for a in guideline] for guideline in guidelines]

guidelines_all = [cvtGL2Coords(guidelines) for guidelines in guidelines_all]

N = len(imageName_all)

In [28]:
pbin = '/home/loitg/ssd/data/temp3/protobuf_unify_old'
imagesin = '/home/loitg/ssd/data/temp3/images_unify_old'
# hinhphang1 = '/home/loitg/workspace/dewarp/hinhphang1'
imagestemp = '/home/loitg/workspace/backbone/imagestemp/'

fn = '650000000002571.85b265c4-35fc-4fe0-a240-d9672390ea54.636615750414373359.jpg'
image = cv2.imread(os.path.join(imagesin, fn)) # hinh cong tu nhien
pb_response = readPb(os.path.join(pbin, fn.replace('jpg', 'pb') ))
word_bounds = get_document_bounds(pb_response, FeatureType.WORD)
boxes = [bound2points(bound) for bound in word_bounds]

mask_word = np.zeros(image.shape[:2], dtype=np.uint8) # mask cong tu nhien
# direction_map_x = np.zeros(image.shape[:2], dtype=np.uint8)
# direction_map_y = np.zeros(image.shape[:2], dtype=np.uint8)
# for box in boxes:
#     cv2.fillPoly(mask_word, np.expand_dims(np.array(box, dtype=np.int32), axis=0), 1)
#     d = Coord(*box[2])-Coord(*box[1])
#     cv2.fillPoly(direction_map_x, np.expand_dims(np.array(box, dtype=np.int32), axis=0), d.x)
#     cv2.fillPoly(direction_map_y, np.expand_dims(np.array(box, dtype=np.int32), axis=0), d.y)
        
# direction_map = np.stack([direction_map_y, direction_map_x], axis=2) # direction_map tu nhien

In [36]:
import json
import cv2

for i in range(N):
    name = imageName_all[i]
    if name != fn:
        continue
    image = cv2.imread(os.path.join(imagesin, fn))
    
    
    gls = guidelines_all[i]
    print(gls)
    a_gls = alignGuidelines(gls)
    
    
    print(gls)

[]


ZeroDivisionError: float division by zero

In [35]:
name

'650000000180669.eba6d3b6-2a01-460c-aa31-41600fc7d45b.636617310881658920.jpg'

In [26]:
####### cong tu nhien =>align=> 
control_points = gls2arr(gls)
dst_points = gls2arr(a_gls)

xx0, yy0 = createStandardGrid(image.shape)
grid_shape = xx0.shape
xx0 = xx0.reshape(-1)
yy0 = yy0.reshape(-1)

standargrid = np.stack([yy0,xx0], axis=1)

newgrid = mls(control_points, dst_points, standargrid)
xx1 = newgrid[:,1]
yy1 = newgrid[:,0]
xx1, yy1 = centralize(xx1, yy1)

mapping = GridMap(np.stack([yy0, xx0], axis=1), np.stack([yy1, xx1], axis=1))

#########
new_image = mapping.transformArrayScalar_bilinear(image)
cv2.imwrite(os.path.join(imagestemp, '%s_flatten.jpg' % (fn)), new_image)
cv2.imwrite(os.path.join(imagestemp, '%s_orgininal.jpg' % (fn)), image)

deviation_field = mapping.deviationFromShape(image.shape)
# deviation_field = drawField(image, deviation_field)
# cv2.imwrite(os.path.join(imagestemp, '%s_deviation_field.jpg' % (fn)), deviation_field)

# new_image, new_mask, new_vmap, F = mapping.transformAll(image, mask_word, direction_map)

# cv2.imwrite(os.path.join(imagestemp, '%s_labeler_mask.jpg' % (fn)), new_mask*255)
# new_vmap = drawField(new_image, new_vmap)
# cv2.imwrite(os.path.join(imagestemp, '%s_labeler_vmap.jpg' % (fn)), new_vmap)
# F = drawField(image, F)
# cv2.imwrite(os.path.join(imagestemp, '%s_labeler_F.jpg' % (fn)), F)

In [27]:
grid = np.moveaxis(np.mgrid[0:image.shape[0]:1, 0:image.shape[1]:1], 0, 2)
new_grid = (grid + deviation_field).astype(int)
np.clip(new_grid[:,:,0], 0,image.shape[0]-1, out=new_grid[:,:,0])
np.clip(new_grid[:,:,1], 0,image.shape[1]-1, out=new_grid[:,:,1])
new_image = np.ndarray(shape=image.shape, dtype=np.uint8)
new_image[new_grid[:,:,0].reshape(-1), new_grid[:,:,1].reshape(-1)] = image[grid[:,:,0].reshape(-1), grid[:,:,1].reshape(-1)]
cv2.imwrite(os.path.join(imagestemp, '%s_reconstruct.jpg' % (fn)), new_image)

True

In [9]:
grid = np.mgrid[0:image.shape[0]:1, 0:image.shape[1]:1]
grid_y, grid_x = grid

In [164]:
new_vmap.shape

(1497, 800, 2)

In [9]:
for fn in imageName_all:
    if fn == '650000000970875.2dd3731c-184a-4fbe-bc8b-e1333a732e72.636613902329249130.jpg':
        print(fn)
        break

In [160]:
np.where(np.isnan(f).sum(axis=1)==2)[0]

array([105, 110, 210, 300, 305])

In [8]:
# control_points = gls2arr(gls)
# dst_points = gls2arr(a_gls)
# points = createStandardGrid((20,20), cell_size=1)
# standargrid = np.stack(points, axis=2).reshape(-1,2)
# newgrid = mls(control_points, dst_points, standargrid)
# ppp(newgrid[:,0], newgrid[:,1], '')